In [1]:
import sqlalchemy as sa
from datetime import timedelta,datetime
import pandas as pd
import numpy as np
import requests
import json
import time
import os
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from shapely.geometry import Point, Polygon
import overpy
import re

In [ ]:
import sqlalchemy as sa
import pandas as pd
import requests
import json

In [ ]:
#Database Credentials
my_db_secret = {
    'drivername': 'postgresql+psycopg2', 
    'host': 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port': '5432', 
    'username': 'shawnmd', 
    'password': '2023!Schulich',
    'database': 'sushmit9_db'
}

#Creating Databse URL
my_db_url = sa.engine.URL.create(
    drivername = my_db_secret['drivername'],
    username = my_db_secret['username'],
    password = my_db_secret['password'],
    host = my_db_secret['host'],
    port = my_db_secret['port'],
    database = my_db_secret['database'])

# Create the SQLAlchemy engine
my_db_engine = sa.create_engine(my_db_url)

## Get data frome API

In [5]:
# Data Source: 	https://data.ontario.ca/api/3/action/datastore_search
api_url = 'https://data.ontario.ca/api/3/action/datastore_search?resource_id=8a89caa9-511c-4568-af89-7f2174b4378c&limit=100'
print(api_url)   

https://data.ontario.ca/api/3/action/datastore_search?resource_id=8a89caa9-511c-4568-af89-7f2174b4378c&limit=100


In [6]:
#Pull the data
data = requests.get(api_url)
data = data.json()
data

{'help': 'https://data.ontario.ca/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 100,
  'records_format': 'objects',
  'resource_id': '8a89caa9-511c-4568-af89-7f2174b4378c',
  'total_estimation_threshold': None,
  'records': [{'_id': 1,
    'report_date': '2021-01-01T00:00:00',
    'previous_day_total_doses_administered': 6252,
    'previous_day_at_least_one': 6242,
    'previous_day_fully_vaccinated': 2,
    'previous_day_3doses': None,
    'total_doses_administered': 32210,
    'total_individuals_at_least_one': '32155',
    'total_individuals_partially_vaccinated': '32144',
    'total_doses_in_fully_vaccinated_individuals': '22',
    'total_individuals_fully_vaccinated': '11',
    'total_individuals_3doses': None},
   {'_id': 2,
    'report_date': '2021-01-02T00:00:00',
    'previous_day_total_doses_administered': 4615,
    'previous_day_at_least_one': 4603,
    'previous_day_fully_vaccinated': 6,
    'previous_day_3dose

In [44]:
record = data['result']['records']
df = pd.DataFrame(record)
df

,_id,report_date,previous_day_total_doses_administered,previous_day_at_least_one,previous_day_fully_vaccinated,previous_day_3doses,total_doses_administered,total_individuals_at_least_one,total_individuals_partially_vaccinated,total_doses_in_fully_vaccinated_individuals,total_individuals_fully_vaccinated,total_individuals_3doses
0,1,2021-01-01T00:00:00,6252,6242,2,None,32210,32155,32144,22,11,None
1,2,2021-01-02T00:00:00,4615,4603,6,None,36825,36706,36693,26,13,None
2,3,2021-01-03T00:00:00,5396,5389,3,None,42221,42095,42079,32,16,None
3,4,2021-01-04T00:00:00,5996,5993,0,None,48217,48087,48072,30,15,None
4,5,2021-01-05T00:00:00,9687,9662,21,None,57904,57747,57712,70,35,None
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,2021-04-06T00:00:00,79584,77714,1888,None,2799632,2419581,2050157,738848,369424,None
96,97,2021-04-07T00:00:00,108012,104944,3056,None,2907644,2524400,2151978,744844,372422,None
97,98,2021-04-08T00:00:00,114080,110106,4028,None,3021724,2634395,2258001,752788,376394,None
98,99,2021-04-09T00:00:00,109855,105625,4301,None,3131579,2739902,2359272,761260,380630,None


New Doses Administered (new_doses)

New Doses for Unvaccinated (new_doses_unvaccinated)

New Doses for people with one vaccine (new_doses_one_vaccine)

New Doses for people with two vaccines (new_doses_two_vaccines)

New Doses for people with three vaccines (new_doses_three_vaccines)


# Feature Engineering 

In [48]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   _id                                          100 non-null    int64  
 1   report_date                                  100 non-null    object 
 2   previous_day_total_doses_administered        100 non-null    int64  
 3   previous_day_at_least_one                    100 non-null    int64  
 4   previous_day_fully_vaccinated                100 non-null    int64  
 5   previous_day_3doses                          100 non-null    int64  
 6   total_doses_administered                     100 non-null    int64  
 7   total_individuals_at_least_one               100 non-null    object 
 8   total_individuals_partially_vaccinated       100 non-null    object 
 9   total_doses_in_fully_vaccinated_individuals  100 non-null    object 
 10  tot

In [46]:
df['previous_day_3doses'] = df['previous_day_3doses'].fillna(0)

df['new_doses'] = df['previous_day_total_doses_administered'].shift(-1).fillna(0)

# total - at least one equal to new dose unvacciinted
df['new_doses_unvaccinated'] = (df['previous_day_total_doses_administered']-df['previous_day_at_least_one']).shift(-1).fillna(0)

df['new_doses_one_vaccine'] = (df['previous_day_at_least_one']- df['previous_day_fully_vaccinated']-df['previous_day_3doses']).shift(-1).fillna(0)

df['new_doses_two_vaccines'] = df['previous_day_fully_vaccinated'].shift(-1).fillna(0)

df['new_doses_three_vaccines'] = df['previous_day_3doses'].shift(-1).fillna(0)
df

,_id,report_date,previous_day_total_doses_administered,previous_day_at_least_one,previous_day_fully_vaccinated,previous_day_3doses,total_doses_administered,total_individuals_at_least_one,total_individuals_partially_vaccinated,total_doses_in_fully_vaccinated_individuals,total_individuals_fully_vaccinated,total_individuals_3doses,new_doses,new_doses_unvaccinated,new_doses_one_vaccine,new_doses_two_vaccines,new_doses_three_vaccines
0,1,2021-01-01T00:00:00,6252,6242,2,0,32210,32155,32144,22,11,None,4615.0,12.0,4597.0,6.0,0.0
1,2,2021-01-02T00:00:00,4615,4603,6,0,36825,36706,36693,26,13,None,5396.0,7.0,5386.0,3.0,0.0
2,3,2021-01-03T00:00:00,5396,5389,3,0,42221,42095,42079,32,16,None,5996.0,3.0,5993.0,0.0,0.0
3,4,2021-01-04T00:00:00,5996,5993,0,0,48217,48087,48072,30,15,None,9687.0,25.0,9641.0,21.0,0.0
4,5,2021-01-05T00:00:00,9687,9662,21,0,57904,57747,57712,70,35,None,12793.0,465.0,11878.0,450.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,2021-04-06T00:00:00,79584,77714,1888,0,2799632,2419581,2050157,738848,369424,None,108012.0,3068.0,101888.0,3056.0,0.0
96,97,2021-04-07T00:00:00,108012,104944,3056,0,2907644,2524400,2151978,744844,372422,None,114080.0,3974.0,106078.0,4028.0,0.0
97,98,2021-04-08T00:00:00,114080,110106,4028,0,3021724,2634395,2258001,752788,376394,None,109855.0,4230.0,101324.0,4301.0,0.0
98,99,2021-04-09T00:00:00,109855,105625,4301,0,3131579,2739902,2359272,761260,380630,None,109250.0,3764.0,101632.0,3854.0,0.0


## Ingest the data into PostgreSQL server

In [ ]:
my_db_secret = {
    'drivername': 'postgresql+psycopg2', 
    'host': 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port': '5432', 
    'username': 'shawnmd', 
    'password': '2023!Schulich',
    'database': 'shawnmd_db'
}

In [ ]:
my_db_url = sa.engine.URL.create(
    drivername = my_db_secret['drivername'],
    username = my_db_secret['username'],
    password = my_db_secret['password'],
    host = my_db_secret['host'],
    port = my_db_secret['port'],
    database = my_db_secret['database']
)

In [ ]:
my_db_engine = sa.create_engine(my_db_url)

In [49]:
df.columns

Index(['_id', 'report_date', 'previous_day_total_doses_administered',
       'previous_day_at_least_one', 'previous_day_fully_vaccinated',
       'previous_day_3doses', 'total_doses_administered',
       'total_individuals_at_least_one',
       'total_individuals_partially_vaccinated',
       'total_doses_in_fully_vaccinated_individuals',
       'total_individuals_fully_vaccinated', 'total_individuals_3doses',
       'new_doses', 'new_doses_unvaccinated', 'new_doses_one_vaccine',
       'new_doses_two_vaccines', 'new_doses_three_vaccines'],
      dtype='object')

In [ ]:
df.to_sql(
    name = 'Moose hunting activity and harvests',
    con = my_db_engine,
    schema = 'final exam',
    if_exists = 'replace',
    index = False,
    method = 'multi',
    dtype = {
        '_id': sa.types.INTEGER,
        'report_date': sa.types.VARCHAR(25),
        'previous_day_total_doses_administered': sa.types.INTEGER,
        'previous_day_at_least_one' : sa.types.INTEGER, 
        'previous_day_fully_vaccinated': sa.types.INTEGER,
        'previous_day_3doses': sa.types.INTEGER,
        'total_doses_administered': sa.types.INTEGER, 
        'total_individuals_at_least_one': sa.types.INTEGER, 
        'total_individuals_partially_vaccinated': sa.types.INTEGER,
        'total_doses_in_fully_vaccinated_individuals': sa.types.FLOAT,
        'total_individuals_fully_vaccinated': sa.types.FLOAT, 
        'total_individuals_3doses': sa.types.FLOAT,
        'new_doses': sa.types.FLOAT,
        'new_doses_unvaccinated': sa.types.FLOAT,
        'new_doses_one_vaccine': sa.types.FLOAT,
        'new_doses_two_vaccines': sa.types.FLOAT,
        'new_doses_three_vaccines': sa.types.FLOAT,

    }
)